In [1]:
import sys,os
sys.path.append('../')
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
setLogLevel("error")

# Set the path to the database
database = Database(os.path.expanduser("../test/database"))

### Load model

In [2]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 0.5*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

### Load the experimental results to be used.

In [4]:
listOfExpRes = database.getExpResults()

### Compute the theory predictions

In [5]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = []
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        txnames = [str(txname) for txname in theoryPrediction.txnames]
        print("------------------------")
        print("Dataset = ", datasetID)  # Analysis name
        print("TxNames = ", txnames)
        print("Theory Prediction = ", theoryPrediction.xsection)  # Signal cross section
        print("Condition Violation = ", theoryPrediction.conditions)  # Condition violation values

        # Get the corresponding upper limit:
        print("UL for theory prediction = ", theoryPrediction.upperLimit)

        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)
        # Compute likelihoods for EM-type results:
        if dataset.getType() == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print('L_BSM, L_SM, L_max = %1.3E, %1.3E, %1.3E' % (theoryPrediction.likelihood(),
                  theoryPrediction.lsm(), theoryPrediction.lmax()))
        if r > rmax:
            rmax = r
            bestResult = expResult.globalInfo.id
        allPredictions.append(theoryPrediction)


 Theory Predictions and Constraints:

 ATLAS-CONF-2013-037 
------------------------
Dataset =  SRbC3
TxNames =  ['T1qqtt', 'T1tttt', 'T1bbtt']
Theory Prediction =  4.67E-05 [pb]
Condition Violation =  None
UL for theory prediction =  4.00E-01 [fb]
r = 1.169E-01
L_BSM, L_SM, L_max = 1.886E-02, 2.067E-02, 2.067E-02

 ATLAS-SUSY-2013-02 
------------------------
Dataset =  None
TxNames =  ['T1']
Theory Prediction =  1.53E+00 [pb]
Condition Violation =  None
UL for theory prediction =  2.33E+07 [fb]
r = 6.560E-05

 ATLAS-SUSY-2013-12 
------------------------
Dataset =  None
TxNames =  ['TChiWZoff']
Theory Prediction =  1.40E+00 [pb]
Condition Violation =  [0.0]
UL for theory prediction =  2.17E+03 [fb]
r = 6.460E-01

 CMS-SUS-13-012 
------------------------
Dataset =  8NJetinf_1000HT1250_200MHTinf
TxNames =  ['T1tttt']
Theory Prediction =  2.61E-06 [pb]
Condition Violation =  None
UL for theory prediction =  5.17E-01 [fb]
r = 5.050E-03
L_BSM, L_SM, L_max = 2.084E-02, 2.061E-02, 2.652E-